In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
seen_df = pd.read_feather("../data/labels.feather")

In [5]:
MODEL = "climatebert"
y_prefix = "2 -"

#unseen_df = pd.read_feather("data/0_unlabelled_documents.feather")
unseen_df = pd.read_feather("../data/oa.feather")

def get_categories(y_prefix, unseen_df, min_samples=15):
    pred = np.load(f"../cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
    ids = np.load(f"../cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
    cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
    cols = [x for x in cols if seen_df[x].sum()>min_samples]
    pred_df = pd.DataFrame({"id": ids})
    if y_prefix=='INCLUDE':
        pred_df['INCLUDE']=pred
    else:
        for i, cname in enumerate(cols):
            pred_df[cname]=pred[:,i]
    df = pd.concat([
        unseen_df.merge(pred_df, how='inner'),
        unseen_df.merge(seen_df[['id']+cols], how='inner')
    ]).reset_index(drop=True)
    print(df.shape)
    return df

print(unseen_df.shape)
unseen_df = get_categories("2 -", unseen_df)
unseen_df[['id','2 - 0. Mitigation','2 - 1. Adaptation']].to_feather('~/ownCloud/climate-literature/mitigation_adaptation.feather')
unseen_df.head()

(1026371, 6)
(846944, 8)


,id,title,abstract,publication_year,type,doi,2 - 0. Mitigation,2 - 1. Adaptation
0,W9999146,Industrial Applications of Plant-Wide Equation...,Mathematical models of industrial processes ha...,2011.0,book-chapter,https://doi.org/10.1016/b978-0-12-380985-8.000...,0.999565,0.000958
1,W999906588,Climate change scenarios and behavioural chang...,In constructing normative scenarios images of ...,2008.0,None,None,0.999658,0.001153
2,W99983702,A Mechanism for Decadal Climate Variability,We describe in this paper a mechanism for deca...,1996.0,book-chapter,https://doi.org/10.1007/978-3-662-03291-6_6,0.999577,0.116017
3,W999802905,Why responses to dramatic climate change are i...,Climate change is proceeding at an unprecedent...,2011.0,journal-article,https://doi.org/10.1093/czoolo/57.3.350,0.992068,0.957751
4,W999738044,"CHANGES IN ATMOSPHERIC CH4, O3, NO2, SO2 CONCE...",Ab s t r a c t . Changes in atmospheric concen...,2014.0,None,None,0.999660,0.001167


In [6]:
MODEL = "climatebert"

#unseen_df = pd.read_feather("data/0_unlabelled_documents.feather")
unseen_df = pd.read_feather("../data/oa.feather")

print(unseen_df.shape)
unseen_df = get_categories("INCLUDE", unseen_df)

unseen_df[['id','INCLUDE']].reset_index(drop=True).to_feather('../data/relevance.feather')

unseen_df = unseen_df[unseen_df['INCLUDE']>=0.5]
unseen_df.head()

(1026371, 6)
(845914, 7)


,id,title,abstract,publication_year,type,doi,INCLUDE
5,W999667810,Modelling Stand Dynamics after Selective Loggi...,Abstract: Forest degradation and biomass damag...,2012.0,None,None,0.874684
6,W999644455,"Mathematics, Politics, and Greenhouse Gas Inte...",While reading the newspaper one morning in spr...,2003.0,journal-article,https://doi.org/10.5951/mt.96.9.0646,0.975826
10,W999485533,The Contribution of Rural Development Programm...,"The Health Check reform, reinforcing the Commo...",2012.0,book-chapter,https://doi.org/10.1007/978-3-642-22266-5_23,0.993647
25,W998914803,Negotiating greenhouse abatement and the theor...,The initial qualified commitment of Annex 1 co...,1999.0,book-chapter,https://doi.org/10.1007/978-94-015-9169-0_3,0.992480
52,W998247206,Is the introduction of a carbon tax a ‘teachab...,With the arrival of the carbon tax earlier thi...,2012.0,None,None,0.989788


In [7]:
unseen_df[unseen_df['id'].isin(seen_df['id'])]

,id,title,abstract,publication_year,type,doi,INCLUDE
844391,W97488302,More pain at the diesel pump? An econometric c...,The recent revision of the European Commission...,2014.0,posted-content,None,1.0
844394,W869671215,An Evaluation of Optimal Application of Govern...,The major purpose of this paper is (1) to find...,2005.0,None,None,1.0
844400,W653696280,Carbon Prices for The Next Hundred Years,This paper examines the socially optimal prici...,2018.0,journal-article,https://doi.org/10.1111/ecoj.12436,1.0
844403,W597825501,FUEL CONSUMPTION AND CO2-RATES FOR INLAND VESSELS,There has recently been much debate on policie...,2004.0,journal-article,https://doi.org/10.2495/ut040621,1.0
844404,W573997129,Understanding the contribution of non-carbon d...,"Abstract In 2010, the combined emissions of me...",2015.0,journal-article,https://doi.org/10.1016/j.gloenvcha.2015.04.010,1.0
...,...,...,...,...,...,...,...
845897,W1538565923,Modelling lifecycle carbon emissions of broadb...,Broadband access systems use electricity gener...,2015.0,proceedings-article,https://doi.org/10.1109/noc.2015.7238614,1.0
845901,W1506098053,Assessing local climate action plans for publi...,Climate change presents a complex environmenta...,2015.0,journal-article,https://doi.org/10.1080/13549839.2015.1038227,1.0
845902,W1485471337,Research and practice on incorporating mobile ...,This paper describes the progress of the resea...,2015.0,journal-article,https://doi.org/10.1063/1.4927001,1.0
845906,W131351029,Australia's Renewable Energy Certificate System,Publisher Summary Concern about climate change...,2003.0,book-chapter,https://doi.org/10.1016/b978-008044276-1/50183-5,1.0


In [8]:
   
for y_prefix in ["4 -", "5 -", "8 -", "10 -", "17 -", "19 -"]:
    unseen_df = get_categories(y_prefix, unseen_df)
unseen_df.head()

(84753, 12)
(84788, 25)
(84815, 32)
(84862, 34)
(84908, 37)
(84990, 39)


,id,title,abstract,publication_year,type,doi,INCLUDE,4 - 1. Economic instruments,4 - 2. Regulatory Instruments,"4 - 3. Information, education and training",...,8 - 05. Transport,8 - 06. Waste,8 - 15. Cross-sectoral,10 - 3. Quantitative,10 - 4. Qualitative,17 - 0. Supranational and international,17 - 1. National,17 - 2. Sub-national,19 - 0. Ex-post,19 - 1. Ex-ante
0,W999667810,Modelling Stand Dynamics after Selective Loggi...,Abstract: Forest degradation and biomass damag...,2012.0,None,None,0.874684,0.968883,0.004977,0.011385,...,0.016040,0.014718,0.033181,0.985738,0.003196,0.996058,0.449868,0.002271,0.081429,0.793856
1,W999644455,"Mathematics, Politics, and Greenhouse Gas Inte...",While reading the newspaper one morning in spr...,2003.0,journal-article,https://doi.org/10.5951/mt.96.9.0646,0.975826,0.010938,0.055075,0.015759,...,0.009628,0.005163,0.944514,0.639513,0.235026,0.007454,0.784754,0.007848,0.958106,0.032210
2,W999485533,The Contribution of Rural Development Programm...,"The Health Check reform, reinforcing the Commo...",2012.0,book-chapter,https://doi.org/10.1007/978-3-642-22266-5_23,0.993647,0.291385,0.436876,0.014692,...,0.014589,0.015739,0.045755,0.624258,0.240851,0.007159,0.997879,0.012517,0.970276,0.189594
3,W998914803,Negotiating greenhouse abatement and the theor...,The initial qualified commitment of Annex 1 co...,1999.0,book-chapter,https://doi.org/10.1007/978-94-015-9169-0_3,0.992480,0.941576,0.014337,0.010090,...,0.009346,0.008797,0.983416,0.527487,0.324508,0.996055,0.009479,0.004130,0.256601,0.991607
4,W998247206,Is the introduction of a carbon tax a ‘teachab...,With the arrival of the carbon tax earlier thi...,2012.0,None,None,0.989788,0.982208,0.006297,0.020769,...,0.003302,0.006184,0.594865,0.438107,0.440414,0.005715,0.008158,0.008924,0.006535,0.990173


In [9]:
unseen_df[unseen_df['id'].isin(seen_df['id'])]

,id,title,abstract,publication_year,type,doi,INCLUDE,4 - 1. Economic instruments,4 - 2. Regulatory Instruments,"4 - 3. Information, education and training",...,8 - 05. Transport,8 - 06. Waste,8 - 15. Cross-sectoral,10 - 3. Quantitative,10 - 4. Qualitative,17 - 0. Supranational and international,17 - 1. National,17 - 2. Sub-national,19 - 0. Ex-post,19 - 1. Ex-ante
84210,W3020205819,Quantifying air quality co-benefits of climate...,Abstract The overlap in sources of greenhouse ...,2020.0,journal-article,https://doi.org/10.1007/s10584-020-02685-7,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.985208,0.003390,0.999162,0.093717,0.005941,0.020151,0.995050
84211,W3020205819,Quantifying air quality co-benefits of climate...,Abstract The overlap in sources of greenhouse ...,2020.0,journal-article,https://doi.org/10.1007/s10584-020-02685-7,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,NaN,0.000000,0.999162,0.093717,0.005941,0.020151,0.995050
84212,W3020205819,Quantifying air quality co-benefits of climate...,Abstract The overlap in sources of greenhouse ...,2020.0,journal-article,https://doi.org/10.1007/s10584-020-02685-7,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.985208,0.003390,1.000000,NaN,0.000000,0.020151,0.995050
84213,W3020205819,Quantifying air quality co-benefits of climate...,Abstract The overlap in sources of greenhouse ...,2020.0,journal-article,https://doi.org/10.1007/s10584-020-02685-7,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,NaN,0.000000,1.000000,NaN,0.000000,0.020151,0.995050
84214,W2511570510,Role of planning strategies in success/failure...,Joint Forest Management (JFM) relies on low co...,2005.0,journal-article,None,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.933187,0.018481,0.078173,0.005841,0.954213,0.964837,0.106008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84985,W1538565923,Modelling lifecycle carbon emissions of broadb...,Broadband access systems use electricity gener...,2015.0,proceedings-article,https://doi.org/10.1109/noc.2015.7238614,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000
84986,W1506098053,Assessing local climate action plans for publi...,Climate change presents a complex environmenta...,2015.0,journal-article,https://doi.org/10.1080/13549839.2015.1038227,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
84987,W1485471337,Research and practice on incorporating mobile ...,This paper describes the progress of the resea...,2015.0,journal-article,https://doi.org/10.1063/1.4927001,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000
84988,W131351029,Australia's Renewable Energy Certificate System,Publisher Summary Concern about climate change...,2003.0,book-chapter,https://doi.org/10.1016/b978-008044276-1/50183-5,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000


In [13]:
for c in unseen_df.select_dtypes(np.float64).columns:
    unseen_df[c] = unseen_df[c].round(2).astype(np.float16)
    
unseen_df["title"] = unseen_df["title"].astype("string")
unseen_df["abstract"] = unseen_df["abstract"].astype("string")
    
unseen_df["publication_year"] = unseen_df["publication_year"].astype("Int16")
unseen_df.reset_index(drop=True).to_feather("../data/annotated_data.feather", compression="zstd")